<a href="https://colab.research.google.com/github/Nandini37/Capstone-Project/blob/main/Capstone_Project_Shipping_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ETL for Script

In [ ]:
# # Read the Excel file

# UPSGround = pd.read_excel("D:/assignment/4th sem/Solution/UPSGround.xlsx")
# USPSGround = pd.read_excel('D:/assignment/4th sem/Solution/USPSGround.xlsx')
# FedEx2day = pd.read_csv('D:/assignment/4th sem/Solution/2DayCounter_Fedex.csv',encoding='latin1')
# FedExGround = pd.read_csv('D:/assignment/4th sem/Solution/FHDCounter.csv',encoding='latin1')


# # print(FedExGround.head())
# # print(FedEx2day.head())
# # print(USPSGround.head())
# # print(UPSGround.head())


# UPSGround = pd.melt(UPSGround, id_vars=['WeightStartOz', 'WeightEndOz'], var_name='Zone', value_name='Value')
# UPSGround['Carrier'] = 'UPS'
# UPSGround['CarrierService'] = 'UPSGround'

# def transform_df(df, additional_columns):
#         df_melted = pd.melt(df, id_vars=['WeightStartOz', 'WeightEndOz'], var_name='Zone', value_name='Value')
#        # Add additional columns
#         for col_name, col_value in additional_columns.items():
#             df_melted[col_name] = col_value

#         return df_melted



# FedEx2day= transform_df(FedEx2day, additional_columns={'Carrier': 'FedEx', 'CarrierService': 'FedEx2day'})
# FedExGround = transform_df(FedEx2day, additional_columns={'Carrier': 'FedEx', 'CarrierService': 'FedExGround'})
# UPSGround = transform_df(FedEx2day, additional_columns={'Carrier': 'UPS', 'CarrierService': 'UPSGround'})
# USPSGround = transform_df(FedEx2day, additional_columns={'Carrier': 'USPS', 'CarrierService': 'USPSGround'})



# df = FedEx2day.append([FedExGround,UPSGround,USPSGround])



# from sqlalchemy import create_engine



# # Define your SQL Server connection parameters
# server = 'LAPTOP-2MROT4G8\SQLEXPRESS'
# database = 'master'
# table_name = 'Ratecard'

# # Create a connection string using Trusted Connection
# connection_string = f"mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"

# # Create an SQLAlchemy engine
# engine = create_engine(connection_string)

# # Load DataFrame into SQL Server table
# df.to_sql(table_name, engine, if_exists='replace', index=False)

# print("DataFrame successfully loaded into SQL Server table.")


# Workflow:

---



1.   Input values of l,b,h,actual weight, origin zipcode and Destination zipcode
2.   Calculate dim weight
3.   Calculate Billable weight
4.   Find zone mapping from origin zipcode
5.   Print Zone and Billable weight
6.   Give the cheapest carrier rate
7.   Show a table with alternative carrier and rate




Function:



*   Dim Weight  ✅
*   Billable Weight ✅
*   Zone from Origin Zipcode ✅
*   Find rate using bill weight and zone



In [ ]:
# Function to calculate shipping cost
def calculate_dim_weight(length, height, width):
    # Calculate dimensional weight in ounces
    dimensional_weight = ((length * height * width) / 166) * 16
    return dimensional_weight

def calcuate_billable_weight(dimensional_weight, weight):
    # Calculate billable weight
     billable_weight = max(math.ceil(dimensional_weight), math.ceil(weight))
     return billable_weight

In [ ]:
import requests
import pandas as pd
import json


def get_zone_chart(zip_code, shipping_date):
    # Construct the URL with the given ZIP code and shipping date
    url = f"https://postcalc.usps.com/DomesticZoneChart/GetZoneChart?zipCode3Digit={zip_code}&shippingDate={shipping_date}"

    # Optional headers (these can be customized if necessary)
    headers = {
        'authority': 'postcalc.usps.com',
        'method': 'GET',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    }

    # Send the GET request to the USPS API
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raises an HTTPError if the HTTP request failed
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Handle HTTP errors
        return None
    except Exception as err:
        print(f"Other error occurred: {err}")  # Handle other errors
        return None

    # Parse the JSON data if the request was successful
    try:
        data = response.json()
    except json.JSONDecodeError:
        print("Failed to parse JSON response.")
        return None

    # Extract the relevant part of the JSON data
    try:
        columns_data0 = data['Column0']
        columns_data1 = data['Column1']
        columns_data2 = data['Column2']
        Zip5Digit = data['Zip5Digit']

        # Convert columns data to DataFrames
        columns_data0 = pd.DataFrame(columns_data0)
        columns_data1 = pd.DataFrame(columns_data1)
        columns_data2 = pd.DataFrame(columns_data2)
        # Zip5Digit = pd.DataFrame(Zip5Digit)  # Uncomment this if needed for Zip5Digit

        # Combine DataFrames along the rows
        combined_df = pd.concat([columns_data0, columns_data1, columns_data2], axis=0)

        # Make a copy of the combined DataFrame
        df = combined_df.copy()

        return df

    except KeyError as e:
        print(f"Key error: {e} not found in the response data.")
        return None


In [ ]:
# Example usage
zip_code = "700"  # Example ZIP code
shipping_date = "8/12/2024"  # Example shipping date in MM/DD/YYYY format

df = get_zone_chart(zip_code, shipping_date)


# Function to expand ZIP code ranges
def expand_zipcodes(zip_code):
    if '---' in zip_code:
        start, end = zip_code.split('---')
        return [str(i).zfill(3) for i in range(int(start), int(end) + 1)]
    else:
        return [zip_code]

# Expand the ZIP codes based on the existing DataFrame
expanded_zipcodes = []

for i, row in df.iterrows():
    zip_codes = expand_zipcodes(row['ZipCodes'])
    for zip_code in zip_codes:
        expanded_zipcodes.append({'ZipCode': zip_code, 'Zone': row['Zone']})

# Create a new DataFrame with expanded ZIP codes
zipZoneMapping = pd.DataFrame(expanded_zipcodes)

,ZipCode,Zone
0,005,6
1,006,7
2,007,7
3,008,7
4,009,7
...,...,...
738,783,4
739,784,4
740,785,4
741,786,4
